In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import numpy
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from collections import OrderedDict

# %config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
# %matplotlib inline

In [115]:
# Se usaran los sets normalizados bajo columnas y TITULO, vease la mejora en el error.
train = pd.read_csv('../normalized2_train.csv')
test = pd.read_csv('../normalized2_test.csv')

const = ['antiguedad', 'banos', 'garages', 'metroscubiertos', 'metrostotales', 'metros']
every_column_except_y= [col for col in train.columns if col not in ['preciomt2', 'precio','id', 'logpreciomt2', 'fecha', 'descripcion', 'titulo', 'logprecio']]
columns = [col for col in train[every_column_except_y].columns if col not in const]
# columns
train.loc[:, columns] = train[columns].fillna(0)
for column in columns:
    train[column] = train[column].astype('int32')



In [104]:
train[every_column_except_y].dtypes

antiguedad                    float64
banos                         float64
centroscomercialescercanos      int32
ciudad_0                        int32
ciudad_1                        int32
ciudad_2                        int32
ciudad_3                        int32
ciudad_4                        int32
ciudad_5                        int32
ciudad_6                        int32
ciudad_7                        int32
ciudad_8                        int32
ciudad_9                        int32
ciudad_10                       int32
escuelascercanas                int32
garages                       float64
gimnasio                        int32
habitaciones                    int32
metroscubiertos               float64
metrostotales                 float64
piscina                         int32
provincia_0                     int32
provincia_1                     int32
provincia_2                     int32
provincia_3                     int32
provincia_4                     int32
provincia_5 

In [122]:
from sklearn.model_selection import train_test_split

xg_reg = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)
every_column_except_y= [col for col in train.columns if col not in ['preciomt2', 'precio','id', 'logpreciomt2', 'fecha', 'descripcion', 'titulo', 'logprecio']]
# every_column_except_y = [col for col in train.columns if col in ['metros', 'antiguedad']]
X, y = train.loc[:,every_column_except_y], train['logprecio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [123]:

xg_reg.fit(X_train,y_train)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:00:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=6, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
             silent=None, subsample=0.8, verbosity=1)

In [121]:
pred= xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))
# pred
rmse

0.31702774309820325

In [46]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [397]:
# rmse_cv(xg_reg).mean() 

In [111]:
OrderedDict(sorted(xg_reg.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('metrostotales', 3599),
             ('metroscubiertos', 3022),
             ('metros', 2827),
             ('antiguedad', 2251),
             ('habitaciones', 1181),
             ('banos', 937),
             ('garages', 613),
             ('tipodepropiedad_4', 550),
             ('provincia_1', 548),
             ('ciudad_6', 546),
             ('provincia_3', 536),
             ('ciudad_5', 524),
             ('fh2', 521),
             ('provincia_4', 511),
             ('ciudad_8', 473),
             ('provincia_2', 469),
             ('provincia_5', 464),
             ('ciudad_4', 435),
             ('fh1', 434),
             ('ciudad_9', 425),
             ('fh5', 418),
             ('ciudad_7', 393),
             ('ciudad_10', 390),
             ('tipodepropiedad_3', 380),
             ('fh15', 338),
             ('centroscomercialescercanos', 330),
             ('fh7', 325),
             ('fh18', 316),
             ('ciudad_3', 307),
             ('fh12', 285),
   

In [112]:
xg_reg.fit(train[every_column_except_y], train['logpreciomt2'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[13:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=6, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
             silent=None, subsample=0.8, verbosity=1)

In [113]:
pred = xg_reg.predict(test[every_column_except_y])
final = test[['id']]
final.loc[:, 'target'] = test.metros * np.expm1(pred)
final.set_index('id').to_csv('../prediction_xgboost.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
